In [ ]:
!conda install tqdm -y

In [1]:
# set up the big query connector
# must be on python3
# Ref: https://cloud.google.com/bigquery/docs/visualize-jupyter
%load_ext google.cloud.bigquery
import tqdm
import numpy as np

In [2]:
%bigquery df
select
   federal_action_obligation,
   total_dollars_obligated,
   base_and_exercised_options_value,
   current_total_value_of_award,
   base_and_all_options_value,
   potential_total_value_of_award,
   sam_exception,
   recipient_parent_duns,
   recipient_congressional_district,
   primary_place_of_performance_congressional_district,
   number_of_actions,
   naics_code,
   number_of_offers_received,
   price_evaluation_adjustment_preference_percent_difference,
   number_of_employees,
   annual_revenue
from `ian-sandbox-221014.ian_usaspending.transactions`
limit 100

,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,sam_exception,recipient_parent_duns,recipient_congressional_district,primary_place_of_performance_congressional_district,number_of_actions,naics_code,number_of_offers_received,price_evaluation_adjustment_preference_percent_difference,number_of_employees,annual_revenue
0,37000.0,None,0.0,None,0.0,None,None,079562245,None,00,1.0,561920.0,0,0,0,0.000000e+00
1,0.0,None,0.0,None,0.0,None,None,000000000,None,None,1.0,NaN,0,0,0,0.000000e+00
2,0.0,None,0.0,None,0.0,None,None,000000000,None,None,1.0,NaN,0,0,0,0.000000e+00
3,87301.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
4,287295.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
5,139192.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
6,246284.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
7,3999150.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
8,200318.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00
9,132497.0,None,0.0,None,0.0,None,None,079253761,AK00,00,1.0,514199.0,0,0,0,0.000000e+00


In [3]:
# running the big query jupyter magic dumps the data into a pandas dataframe!
type(df)

pandas.core.frame.DataFrame

In [ ]:
# DO ALL THE DATA SCIENCE
# Example courtesy of Google: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/cloudmle/cloudmle.ipynb

# Steps
# feature/target engineering
# create hold out set
# cross validation
  # feature selection, dimensionality reduction, etc.
  # model training (using ML Engine)
  # hyperparameter optimization
  # trained model evaluation
  # model selection
# final model evaluation against hold out set
# deploy model to ML Engine
# score new data against deployed model using ML Engine

In [7]:
df[df["number_of_employees"]> 0].head()

,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,sam_exception,recipient_parent_duns,recipient_congressional_district,primary_place_of_performance_congressional_district,number_of_actions,naics_code,number_of_offers_received,price_evaluation_adjustment_preference_percent_difference,number_of_employees,annual_revenue
12,95000.0,None,0.0,None,0.0,None,None,018533823,CA09,00,1.0,511210.0,0,0,80,1.586700e+07
13,0.0,None,0.0,None,60000.0,None,None,798444600,CO03,None,NaN,511140.0,0,None,2,2.000000e+05
15,26486.0,None,0.0,None,0.0,None,None,107939357,VA10,00,1.0,421430.0,0,0,850,9.000000e+09
16,0.0,None,0.0,None,0.0,None,None,612130948,AL01,00,1.0,334119.0,1,0,180,2.730729e+07
17,29166.0,None,0.0,None,0.0,None,None,612130948,AL01,00,1.0,334119.0,1,0,180,2.730729e+07


In [ ]:
num_cols = df._get_numeric_data().columns
num_cols = num_cols.tolist()
float_cols = [c for c in num_cols if df[c].dtype == 'float64']
num_cols.remove('naics_code')
log1p_cols = [
    'federal_action_obligation',
    'base_and_exercised_options_value',
    'current_total_value_of_award',
    'base_and_all_options_value',
    'potential_total_value_of_award',
    'number_of_actions',
    'number_of_offers_received',
    'number_of_employees',
    'annual_revenue'
]
for col in tqdm.tqdm_notebook(log1p_cols):
    print(col)
    df.loc[:, col] = np.log1p(df[col].clip(lower=0))